In [30]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Membaca file CSV
data = pd.read_csv("D:/KULIAH/SEMESTER 5/EKONOMETRIKA/UMR INDONESIA.csv", sep = ';')
data = pd.DataFrame(data)
data.dropna()
data.head()


,TAHUN,UMR,INFLASI,JUMLAH PENDUDUK,ANGKATAN KERJA,BEKERJA,PENGANGGURAN,PERTUMBUHAN EKONOMI
0,1997,135000,11.10%,204.000.000,89.230.978,85.047.007,4.183.971,6.5%
1,1998,150900,77.60%,207.000.000,92.337.801,87.292.541,5.045.260,0
2,1999,175400,2.00%,211.000.000,94.847.178,88.816.859,6.030.319,0.79%
3,2000,216500,9.40%,214.000.000,95.650.961,89.837.730,5.813.231,3.6%
4,2001,290500,12.55%,217.000.000,98.812.448,90.807.417,8.005.031,4.9%


In [31]:
data.dtypes

TAHUN                   int64
UMR                     int64
INFLASI                object
JUMLAH PENDUDUK        object
ANGKATAN KERJA         object
BEKERJA                object
PENGANGGURAN           object
PERTUMBUHAN EKONOMI    object
dtype: object

In [32]:
# Ganti NaN sementara dengan string kosong (jika ada NaN)
data["INFLASI"] = data["INFLASI"].fillna("")

In [33]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Melakukan uji ADF pada kolom UMR
result = adfuller(data['UMR'])

# Menampilkan hasil
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print(f'   {key}: {value}')

# Interpretasi hasil
if result[1] < 0.05:
    print("Data stasioner (tolak hipotesis nol)")
else:
    print("Data tidak stasioner (gagal tolak hipotesis nol)")

ADF Statistic: 2.0480182954597628
p-value: 0.9987321883558152
Critical Values:
   1%: -3.9240193847656246
   5%: -3.0684982031250003
   10%: -2.67389265625
Data tidak stasioner (gagal tolak hipotesis nol)


In [34]:
result

(2.0480182954597628,
 0.9987321883558152,
 9,
 16,
 {'1%': -3.9240193847656246, '5%': -3.0684982031250003, '10%': -2.67389265625},
 421.3959501543679)

In [35]:
data.columns

Index(['TAHUN', 'UMR', 'INFLASI', 'JUMLAH PENDUDUK', 'ANGKATAN KERJA',
       'BEKERJA', 'PENGANGGURAN', 'PERTUMBUHAN EKONOMI'],
      dtype='object')

In [36]:
# Langkah 1: Membersihkan nilai di kolom object
for col in data.columns:
    if data[col].dtypes == 'object': 
        data[col] = (
            data[col]
            .astype(str)  
            .str.replace('%', '', regex=False)  # Hapus simbol persem
            .str.replace('$', '', regex=False)  # Hapus simbol dollar
            .str.replace('.', '', regex=False)  # Hapus tanda titik
            .str.strip()  # Hapus spasi ekstra
            .replace('', np.nan)  # Ganti string kosong dengan NaN
        )

# Langkah 2: Konversi ke float
for col in data.columns:
    if col not in ["TAHUN", "UMR"]:  
        try:
            data[col] = data[col].astype(float)
        except ValueError as e:
            print(f"Error saat mengonversi kolom {col}: {e}")
            print("Nilai yang bermasalah:")
            print(data[col].unique())

# Output hasil
print(data.dtypes)
print(data)

TAHUN                    int64
UMR                      int64
INFLASI                float64
JUMLAH PENDUDUK        float64
ANGKATAN KERJA         float64
BEKERJA                float64
PENGANGGURAN           float64
PERTUMBUHAN EKONOMI    float64
dtype: object
    TAHUN      UMR  INFLASI  JUMLAH PENDUDUK  ANGKATAN KERJA      BEKERJA  \
0    1997   135000   1110.0      204000000.0      89230978.0   85047007.0   
1    1998   150900   7760.0      207000000.0      92337801.0   87292541.0   
2    1999   175400    200.0      211000000.0      94847178.0   88816859.0   
3    2000   216500    940.0      214000000.0      95650961.0   89837730.0   
4    2001   290500   1255.0      217000000.0      98812448.0   90807417.0   
5    2002   362700   1003.0      220000000.0     100779270.0   91647166.0   
6    2003   414700    516.0      223000000.0     102750092.0   92810791.0   
7    2004   458500    640.0      225000000.0     103973387.0   93722036.0   
8    2005   507697   1711.0      228000000.0 

In [38]:
import pandas as pd
from statsmodels.tsa.stattools import coint

data = pd.DataFrame(data)

# Uji ko-integrasi antara INFLASI dan JUMLAH_PENDUDUK
score, p_value, _ = coint(data['INFLASI'], data['JUMLAH PENDUDUK'])

# Menampilkan hasil uji ko-integrasi
print(f'Uji Ko-integrasi p-value: {p_value}')
if p_value < 0.05:
    print("Tolak hipotesis nol: Ada ko-integrasi")
else:
    print("Gagal tolak hipotesis nol: Tidak ada ko-integrasi")


Uji Ko-integrasi p-value: 8.889784337679406e-08
Tolak hipotesis nol: Ada ko-integrasi


In [ ]:
import pandas as pd
import statsmodels.api as sm

X = data[['INFLASI', 'JUMLAH PENDUDUK']]

# Menambahkan konstanta untuk model OLS
X = sm.add_constant(X)

# Model regresi jangka panjang
long_term_model = sm.OLS(data['UMR'], X).fit()

# Menyimpan residual sebagai Error Correction Term (ECT)
data['ECT'] = long_term_model.resid

# Menampilkan hasil
print(long_term_model.summary())
print(data[['UMR', 'ECT']])

                            OLS Regression Results                            
Dep. Variable:                    UMR   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     167.9
Date:                Sat, 23 Nov 2024   Prob (F-statistic):           1.90e-14
Time:                        15:53:09   Log-Likelihood:                -356.56
No. Observations:                  26   AIC:                             719.1
Df Residuals:                      23   BIC:                             722.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -8.436e+06   5.83e+05    -